In [1]:
from landslide4sense.models import Unet
from landslide4sense.data import LandslideDataSet
from landslide4sense.data import LandslideDataSet
from landslide4sense.utils.tools import eval_image
from landslide4sense.models import Unet
from landslide4sense import EPSILON
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
import omegaconf
import matplotlib.pyplot as plt
from torch.nn import Upsample, CrossEntropyLoss
import numpy as np
from tqdm import tqdm

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
cfg = omegaconf.OmegaConf.load("../conf/config.yaml")
cfg

{'data': {'dir': '${hydra:runtime.cwd}/data/', 'train_list': '${hydra:runtime.cwd}/data/train.txt', 'eval_lists_paths': ['${hydra:runtime.cwd}/data/train.txt'], 'eval_names': ['train'], 'test_list': '${hydra:runtime.cwd}/data/valid.txt', 'input_size': [128, 128], 'augmentation': {'module': 'landslide4sense.data.augmentations', 'name': 'transforms'}}, 'model': {'restore_from': None, 'module': 'segmentation_models_pytorch', 'name': 'UnetPlusPlus', 'args': {'in_channels': 14, 'classes': 2}}, 'optimizer': {'restore_from': None, 'module': 'torch.optim', 'name': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0.0005}}, 'loss': {'module': 'segmentation_models_pytorch.losses', 'name': 'DiceLoss', 'args': {'ignore_index': 255, 'mode': 'multiclass'}}, 'train': {'start_epoch': 0, 'steps_per_epoch': 125, 'batch_size': 128, 'num_workers': 2, 'num_steps': 10000, 'num_steps_stop': 10000, 'gpu_id': 0, 'snapshot_dir': '/content/drive/MyDrive/data_science_projects/landslide4sense-2022/models/', 'seed': 42

In [3]:
model_path = "/content/drive/MyDrive/data_science_projects/landslide4sense-2022/models/split_data_aug/epoch_7_val_f1=73.21207127753885.pth"
model_state_dict = torch.load(
    model_path,
    map_location=torch.device('cpu')
)
model = Unet(n_classes=2)
model.load_state_dict(model_state_dict)
interp = torch.nn.Upsample(size=(128, 128), mode="bilinear")

In [4]:
train_dataset = LandslideDataSet(
    "../data/",
    "../data/train_split_val.txt",
    max_iters=1,
    set="labeled",
)

In [5]:
train_loader: DataLoader[LandslideDataSet] = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=cfg.train.num_workers,
    pin_memory=True,
)

In [6]:
model.eval()
all_preds = []

for i, batch in tqdm(
    enumerate(train_loader),
    desc=f"Evaluating on train...",
    total=len(train_loader),
):
    image, label, _, name = batch
    label = label.squeeze().numpy()
    image = image.float().to(device)

    with torch.no_grad():
        pred = model(image)

    pred = interp(F.softmax(pred, dim=1)).detach()
    all_preds.append(pred)

Evaluating on train...:   0%|          | 0/301 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
Evaluating on train...: 100%|██████████| 301/301 [02:24<00:00,  2.08it/s]


In [17]:
thr = 0.5
num_classes = 2

TP_all = np.zeros((num_classes, 1))
FP_all = np.zeros((num_classes, 1))
TN_all = np.zeros((num_classes, 1))
FN_all = np.zeros((num_classes, 1))
F1 = np.zeros((num_classes, 1))
n_valid_sample_all = 0

for pred in tqdm(all_preds):
    pred = (pred[:, 1, :, :] > thr).long()
    pred = pred.squeeze().data.cpu().numpy()
    TP, FP, TN, FN, n_valid_sample = eval_image(
        pred.reshape(-1), label.reshape(-1), num_classes
    )
    TP_all += TP
    FP_all += FP
    TN_all += TN
    FN_all += FN
    n_valid_sample_all += n_valid_sample

OA = np.sum(TP_all) * 1.0 / n_valid_sample_all
for i in range(num_classes):
    P = TP_all[i] * 1.0 / (TP_all[i] + FP_all[i] + EPSILON)
    R = TP_all[i] * 1.0 / (TP_all[i] + FN_all[i] + EPSILON)
    F1[i] = 2.0 * P * R / (P + R + EPSILON)
    if i == 1:
        precision =  P[0] * 100
        recall = R[0] * 100
        f1 = F1[i][0] * 100

print(precision)
print(recall)
print(f1)
F1

100%|██████████| 301/301 [00:00<00:00, 439.24it/s]

0.20927704294256202
1.871809415768576
0.376463685921544


array([[0.98744832],
       [0.00376464]])

In [13]:
n_valid_sample

16384